* EDA2에서 가져온 gameId 파일을 통해 각 게임의 info를 뽑아내고 저장하는 과정 'EDA3'

In [1]:
import numpy as np
import pandas as pd
import requests
import json
from tqdm import tnrange
import time
from sklearn.model_selection import train_test_split

In [2]:
api_key = 'RGAPI-eabfcbf1-25a3-4971-be6c-036dadfe7f42'

In [3]:
gameid_df = pd.read_csv('data/saving_data/match_info_df.csv')
gameid_df

,champion,gameId,lane,platformId,queue,role,season,timestamp
0,117,4381376153,BOTTOM,KR,450,SOLO,13,1589537068352
1,121,4381381692,NONE,KR,900,DUO_SUPPORT,13,1589535788955
2,142,4381075169,NONE,KR,900,DUO_SUPPORT,13,1589528063963
3,50,4379340987,NONE,KR,450,DUO_SUPPORT,13,1589460817881
4,110,4379295423,NONE,KR,450,DUO_SUPPORT,13,1589459523654
5,84,4378199138,MID,KR,420,SOLO,13,1589432499801
6,86,4378174399,TOP,KR,420,SOLO,13,1589430377350
7,266,4376647893,NONE,KR,450,DUO_SUPPORT,13,1589368736523
8,157,4374739537,NONE,KR,450,DUO_SUPPORT,13,1589289812923
9,80,4369746703,NONE,KR,450,DUO_SUPPORT,13,1589114080437


* EDA1~2를 통해 총 384703개의 gameId를 구해낼 수 있었다.
* 하지만 riot api는 2분에 100request 를 처리하는 RATE LIMITS 가 걸려있다.
* 즉, 모든 gameId에 대한 info를 가져오는것은 시간적 문제가 발생한다.
* 따라서, 총 데이터를 셔플한 후 약 10000개의 데이터만 뽑아서 사용하고자 한다.

In [4]:
# 재현성을 위해 시드값 고정해서 약 10000개의 데이터만을 갖는 df 생성
gameid_shuffled_df, _ = train_test_split(gameid_df, test_size=0.974, random_state=42)
gameid_shuffled_df

,champion,gameId,lane,platformId,queue,role,season,timestamp
16304,48,4353346263,JUNGLE,KR,420,NONE,13,1588569216123
166305,164,4367199595,TOP,KR,420,SOLO,13,1589030702498
115981,60,4352098588,JUNGLE,KR,420,NONE,13,1588514577662
96494,202,4358736984,NONE,KR,420,DUO_SUPPORT,13,1588745022341
260115,104,4361173967,NONE,KR,420,DUO_SUPPORT,13,1588837514402
316689,34,4377039377,MID,KR,420,SOLO,13,1589372097554
14274,523,4339975651,MID,KR,420,DUO,13,1588139841997
269992,104,4378064003,JUNGLE,KR,420,NONE,13,1589425548230
48380,110,4374104072,BOTTOM,KR,420,DUO_CARRY,13,1589276891396
238922,2,4317268687,NONE,KR,420,DUO_SUPPORT,13,1587381375511


In [5]:
match_fin = pd.DataFrame()
for i in tnrange(len(gameid_shuffled_df)):    
    
    api_url='https://kr.api.riotgames.com/lol/match/v4/matches/' + str(gameid_shuffled_df['gameId'].iloc[i]) + '?api_key=' + api_key
    r = requests.get(api_url)

    if r.status_code == 200: # response가 정상이면 바로 맨 밑으로 이동하여 정상적으로 코드 실행
        pass

    elif r.status_code == 429:
        while True: # 429error가 끝날 때까지 무한 루프
            if r.status_code == 429:
                time.sleep(3)
                r = requests.get(api_url)         
            elif r.status_code == 200: #다시 response 200이면 loop escape
                break

    elif r.status_code == 503: # 잠시 서비스를 이용하지 못하는 에러
        while True:
            if r.status_code == 503 or r.status_code == 429:
                time.sleep(3)
                r = requests.get(api_url)
            elif r.status_code == 200: # 똑같이 response가 정상이면 loop escape
                break
                
    elif r.status_code == 403: # api갱신이 필요
        print('you need api renewal')
        break

    # 위의 예외처리 코드를 거쳐서 내려왔을 때 해당 코드가 실행될 수 있도록 작성
    mat = pd.DataFrame(list(r.json().values()), index=list(r.json().keys())).T
    match_fin = pd.concat([match_fin,mat])

C:\Users\zone\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  


C:\Users\zone\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [7]:
# 10002건의 match data를 가져왔으며 이를 pkl로 저장해서 이용하자.
match_fin.to_pickle('data/saving_data/match_info.pkl')

In [8]:
# 저장한 데이터 불러오며 확인
data_loading = pd.read_pickle('data/saving_data/match_info.pkl')
data_loading.head(5)

,gameCreation,gameDuration,gameId,gameMode,gameType,gameVersion,mapId,participantIdentities,participants,platformId,queueId,seasonId,status,teams
0,1588569216123,1475,4353346263,CLASSIC,MATCHED_GAME,10.9.318.6057,11,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420,13,NaN,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T..."
0,1589030702498,1404,4367199595,CLASSIC,MATCHED_GAME,10.9.318.6057,11,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420,13,NaN,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ..."
0,1588514577662,1722,4352098588,CLASSIC,MATCHED_GAME,10.9.318.6057,11,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420,13,NaN,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ..."
0,1588745022341,965,4358736984,CLASSIC,MATCHED_GAME,10.9.318.6057,11,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420,13,NaN,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ..."
0,1588837514402,812,4361173967,CLASSIC,MATCHED_GAME,10.9.318.6057,11,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420,13,NaN,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T..."
